`vdom` is Python data object for composing HTML. `mistletoe` is a markdown converter; in this notebook we write a `mistletoe` to `vdom`
renderer.

> There are some outstanding issues about how to handle inline span html.

> __Personal Opinion__: `mistletoe` is easier to extend than `mistune` and `commonmark`.

`vdom`, like `altair`, embed extra data into the notebook that can be consumed later.

This post builds [on previous work that converts `html` to `vdom`](2018-11-01-vdom-from-raw-html.ipynb).

In [1]:
try:
    from .html_to_vdom import html_to_vdom
except:
    from html_to_vdom import html_to_vdom
import mistletoe, vdom, functools, html

In [21]:
mistletoe.span_token._tags.add('svg')
mistletoe.span_token._tags.add('span')

In [29]:
class markdown_to_vdom(mistletoe.html_renderer.HTMLRenderer):
    def __call__(self, str): return self.render(mistletoe.Document(str.splitlines()))
    
    def render_document(self, token):
        result = self.render_inner(token, vdom.div)
        if isinstance(result, str):
            result = self._dom.flush()
        return result
        
    def render_image(self, token):
        return vdom.img(src=token.src, alt=self.render_to_plain(token), title=token.title and self.escape_html(token.title) or '')

    def render_to_plain(self, token):
        if hasattr(token, 'children'):
            token = ''.join(self.render_to_plain(child) for child in token.children)
        else: token = token.content
        return self.render_inner(token)

    def render_link(self, token): return self.render_inner(
        token, vdom.a, href=self.escape_url(token.target), 
        title=token.title and self.escape_html(token.title) or '')

    def render_auto_link(self, token): return self.render_inner(
        token, vdom.a, href=token.mailto and 'mailto:{}'.format(token.target)  or self.escape_url(token.target))

    def render_raw_text(self, token): 
        return self.render_inner(token)

    def render_heading(self, token): 
        return self.render_inner(token, getattr(vdom, F'h{token.level}'))

    def render_block_code(self, token): return vdom.pre(
        vdom.code(html.escape(token.children[0].content), **({
            'class': F"language-{self.escape_html(token.language)}"
        } if token.language else {})))
    
    def render_table(self, token, tag=vdom.table):
        if hasattr(token, 'header'): 
            tag = functools.partial(tag, vdom.thead(self.render_table_row(token.header, is_header=True)))
        return tag(self.render_inner(token, vdom.tbody))

    def render_table_row(self, token, is_header=False): 
        return vdom.tr(*(self.render_table_cell(child, is_header) for child in token.children))

    def render_table_cell(self, token, in_header=False): 
        return self.render_inner(
            token, vdom.th if in_header else vdom.td, align={None: 'left', 0: 'center', 1: 'right'}[token.align])
    
    @staticmethod
    def render_thematic_break(token):  return vdom.hr()
    
    @staticmethod
    def render_line_break(token): return '\n' if token.soft else vdom.br()

    @staticmethod
    def render_html_block(token):  
        return html_to_vdom()(token.content)

    def render_inline_code(self, token): 
        return self.render_inner(token.children[0], vdom.code)

    _dom = None
    
    def render_html_span(self, token):  
        self._dom = (self._dom or html_to_vdom())
        self._dom.feed(token.content)
        if token.content.startswith('</'):
            return self._dom.flush()
        
        return ''
    
    def render_inner(self, token, tag=None, **data):
        tag = tag or (lambda x: x)
        
        if hasattr(token, 'children'):
            for child in token.children:
                tag = functools.partial(tag, self.render(child))
        else:
            tag = functools.partial(tag, token.content)
        render = tag(**data)
        if self._dom and self._dom.buf:
            self._dom.append(render)
            return ''
        return render
    
    render_strong = functools.partialmethod(render_inner, tag=vdom.strong)
    render_emphasis = functools.partialmethod(render_inner, tag=vdom.em)
    render_strikethrough = functools.partialmethod(render_inner, tag=vdom.create_component('del'))        
    render_escape_sequence = functools.partialmethod(render_inner, tag=vdom.span)
    render_paragraph = functools.partialmethod(render_inner, tag=vdom.p)
    render_quote = functools.partialmethod(render_inner, tag=vdom.blockquote)
    render_list_item = functools.partialmethod(render_inner, tag=vdom.li)

    def render_list(self, token, tag=vdom.ul, **data):
        self._dom = (self._dom or html_to_vdom())

        if token.start is not None:
            tag = vdom.ol
            data.update(start=token.start if token.start != 1 else '')
        return self.render_inner(token, tag, **data)


    g =__import__('graphviz').Source('graph {A}')

    markdown_to_vdom()(F"""{''.join(g._repr_svg_().splitlines())}
    """)

In [30]:
class markdown_to_vdom_flex(markdown_to_vdom):
    prepend = None
    def render_list(self, token, tag=vdom.ul, **data):
        setattr(self, 'prepend', getattr(self, 'prepend') or [])
        self.prepend.append(token.children[0].leader if token.children and token.children[0].leader == '-' else None)  
        try:
            if self.prepend[-1]: 
                data.update(style={
                    'width': '100%', 'display': 'flex', 'flex-direction': ['column', 'row'][1]#[sum(map(bool, self.prepend))%2]
                })
                return self.render_inner(token, vdom.div, **data)        
            else: return super().render_list(token, tag, **data)
        finally: self.prepend.pop()        
    
    def render_list_item(self, token, tag=vdom.li, **data):
        if self.prepend[-1]:
            tag = vdom.div
            data.update(style={'flex': '1', 'max-width': '100%'})
        return self.render_inner(token, tag, **data)

    if __name__ == '__main__':
        !ipython -m pytest -- weave.ipynb

In [31]:
    renderers = __import__('pytest').mark.parametrize('renderer', (markdown_to_vdom_flex, markdown_to_vdom))

In [32]:
    @renderers
    def test_renderer(renderer):
        r = markdown_to_vdom()
        r.render(mistletoe.Document("""---
    
    # A table 

    | Tables        | Are           | Cool  |
    | ------------- |:-------------:| -----:|
    | col 3 is      | right-aligned | $1600 |
    | col 2 is      | centered      |   $12 |
    | zebra stripes | are neat      |    $1 |
    
    
    
    * Now we are talaking about some business and that bu Now we are talaking about some business and that bu Now we are talaking about some business and that bu Now we are talaking about some business and that bu Now we are talaking about some business and that bu Now we are talaking about some business and that bu Now we are talaking about some business and that bu Now we are talaking about some business and that bu Now we are talaking about some business and that bu Now we are talaking about some business and that bu Now we are talaking about some business and that bu Now we are talaking about some business and that bu Now we are talaking about some business and that bu Now we are talaking about some business and that bu Now we are talaking about some business and that bu Now we are talaking about some business and that bu Now we are talaking about some business and that bu Now we are talaking about some business and that bu Now we are talaking about some business and that bu Now we are talaking about some business and that bu Now we are talaking about some business and that bu Now we are talaking about some business and that bu Now we are talaking about some business and that bu Now we are talaking about some business and that bu Now we are talaking about some business and that bu 
      * > Does a block quote work in line item

    * asdklfjaskdfj
    
        * Fuck
             
        * A
       
       * B
                
            1. C
            2. D
        * Q
        
    * ###  B
    
    ---

    > Testing things

    ### Howdy\n\n~~Good~~ look buh <span><code>__crap__</code></span>.\n\n<h1>Things</h1>""".splitlines()))

In [33]:
    @renderers
    def test_graphviz_in_markdown_in_vdom(renderer):
        import graphviz
        g = graphviz.Source("graph {A}")
        renderer().render(mistletoe.Document(
        F"""# bUST

        {''.join(g._repr_svg_().splitlines())}""".splitlines()
        ))

markdown_to_vdom()('https://docs.google.com/document/d/1cejQ1SGD4-dNOzsueUq5Fx_V4gKjEgNTDxr6tC__PVw/edit?usp=sharing')._repr_html_()